# Data preprocessing
    - Download data in the server
    - Convert test to sequences.
    - Configure sequences for a RNN model.

## Download data in the server

### Command line in the server
    Path to data:
        cd /home/ubuntu/data/training/keras
    Download dataset: 
        wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    Uncompress it:
        tar -zxvf aclImdb_v1.tar.gz

## Convert test to sequences
    - List of all text files
    - Read files into python
    - Tokenize
    - Create dictionaries to recode
    - Recode tokens into ids and create sentences

In [1]:
#Imports and paths
from __future__ import print_function

import numpy as np

data_path='/home/ubuntu/data/training/keras/aclImdb/'


In [2]:
# Generator of list of files in a folder and subfolders
import os
import shutil
import fnmatch

def gen_find(filepattern, toppath):
    '''
    Generator with a recursive list of files in the toppath that match filepattern 
    Inputs:
        filepattern(str): Command stype pattern 
        toppath(str): Root path
    '''
    for path, dirlist, filelist in os.walk(toppath):
        for name in fnmatch.filter(filelist, filepattern):
            yield os.path.join(path, name)

#Test
#print(gen_find("*.txt", data_path+'train/pos/').next())

In [3]:
def read_sentences(path):
    sentences = []
    sentences_list = gen_find("*.txt", path)
    for ff in sentences_list:
        with open(ff, 'r') as f:
            sentences.append(f.readline().strip())
    return sentences        

#Test
print(read_sentences(data_path+'train/pos/')[0:2])

["I read the book and saw the movie. Both excellent. The movie is diamond among coals during this era. Liebman and Selby dominate the screen and communicate the intensity of their characters without flaw. This film should have made them stars. Shame on the studio for not putting everything they had behind this film. It could have easily been a franchise. Release on DVD is a must and a worthy remake would revive this film. Look for it in your TV guide and if you see it listed, no matter how late, watch it. You won't be disappointed. Do yourself another favor - read the book (same title). It'll blow you away. Times have changed dramatically since those days, or at least we like to think they have.", "Let me first state that while I have viewed every episode of StarTrek at least twice, I do not consider myself a Trekker or Trekkie. Those are people who live in their parents basement and attend conventions wearing costumes with pointed rubber ears. I gave this movie a seven casting aside t

In [4]:
print(read_sentences(data_path+'train/neg/')[0:2])

['What Hopkins does succeed at with this effort as writer and director is giving us a sense that we know absolutely no one in the film. However, perhaps therein lies the problem. His movie has a lot of ambition and his intentions were obviously complex and drawn from very deep within, but it\'s so impersonal. There are no characters. We never know who anyone is, thus there is no investment on our part.<br /><br />It could be about a screenwriter intermingle with his own characters. Is it? Maybe. By that I don\'t mean that Slipstream is ambiguous; I mean that there is no telling. Hopkins\'s film is an experiment. On the face of it, one could make the case that it is about a would-be screenwriter, who at the very moment of his meeting with fate, realizes that life is hit and miss, and/or success is blind chance, as he is hurled into a "slipstream" of collisions between points in time, dreams, thoughts, and reality. Nevertheless, it is so unremittingly cerebral that it leaves no room for 

In [5]:
def tokenize(sentences):
    from nltk import word_tokenize
    print( 'Tokenizing...',)
    tokens = []
    for sentence in sentences:
        tokens += [word_tokenize(sentence)]
    print('Done!')

    return tokens

print(tokenize(read_sentences(data_path+'train/pos/')[0:2]))

Tokenizing...
Done!
[['I', 'read', 'the', 'book', 'and', 'saw', 'the', 'movie', '.', 'Both', 'excellent', '.', 'The', 'movie', 'is', 'diamond', 'among', 'coals', 'during', 'this', 'era', '.', 'Liebman', 'and', 'Selby', 'dominate', 'the', 'screen', 'and', 'communicate', 'the', 'intensity', 'of', 'their', 'characters', 'without', 'flaw', '.', 'This', 'film', 'should', 'have', 'made', 'them', 'stars', '.', 'Shame', 'on', 'the', 'studio', 'for', 'not', 'putting', 'everything', 'they', 'had', 'behind', 'this', 'film', '.', 'It', 'could', 'have', 'easily', 'been', 'a', 'franchise', '.', 'Release', 'on', 'DVD', 'is', 'a', 'must', 'and', 'a', 'worthy', 'remake', 'would', 'revive', 'this', 'film', '.', 'Look', 'for', 'it', 'in', 'your', 'TV', 'guide', 'and', 'if', 'you', 'see', 'it', 'listed', ',', 'no', 'matter', 'how', 'late', ',', 'watch', 'it', '.', 'You', 'wo', "n't", 'be', 'disappointed', '.', 'Do', 'yourself', 'another', 'favor', '-', 'read', 'the', 'book', '(', 'same', 'title', ')', '.'

In [6]:
sentences_trn_pos = tokenize(read_sentences(data_path+'train/pos/'))
sentences_trn_neg = tokenize(read_sentences(data_path+'train/neg/'))
sentences_trn = sentences_trn_pos + sentences_trn_neg


Tokenizing...
Done!
Tokenizing...
Done!


In [7]:
#create the dictionary to conver words to numbers. Order it with most frequent words first
def build_dict(sentences):
#    from collections import OrderedDict

    '''
    Build dictionary of train words
    Outputs: 
     - Dictionary of word --> word index
     - Dictionary of word --> word count freq
    '''
    print( 'Building dictionary..',)
    wordcount = dict()
    #For each worn in each sentence, cummulate frequency
    for ss in sentences:
        for w in ss:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = list(wordcount.values()) # List of frequencies
    keys = list(wordcount) #List of words
    
    sorted_idx = reversed(np.argsort(counts))
    
    worddict = dict()
    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)
    print( np.sum(counts), ' total words ', len(keys), ' unique words')

    return worddict, wordcount


worddict, wordcount = build_dict(sentences_trn)

print(worddict['the'], wordcount['the'])

Building dictionary..
7056193  total words  135098  unique words
2 289298


In [8]:
# 
def generate_sequence(sentences, dictionary):
    '''
    Convert tokenized text in sequences of integers
    '''
    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in ss]

    return seqs

In [9]:
# Create train and test data

#Read train sentences and generate target y
train_x_pos = generate_sequence(sentences_trn_pos, worddict)
train_x_neg = generate_sequence(sentences_trn_neg, worddict)
X_train_full = train_x_pos + train_x_neg
y_train_full = [1] * len(train_x_pos) + [0] * len(train_x_neg)

print(X_train_full[0], y_train_full[0])

[15, 357, 2, 302, 5, 234, 2, 25, 4, 1500, 350, 4, 21, 25, 9, 6492, 847, 34681, 347, 19, 1108, 4, 34706, 5, 31062, 8843, 2, 314, 5, 5965, 2, 3066, 7, 82, 122, 237, 3418, 4, 61, 26, 156, 37, 109, 112, 441, 4, 5240, 33, 2, 1217, 24, 36, 1601, 346, 48, 76, 543, 19, 26, 4, 51, 96, 37, 721, 95, 6, 3238, 4, 23411, 33, 308, 9, 6, 242, 5, 6, 1711, 1022, 66, 10298, 19, 26, 4, 2377, 24, 16, 14, 150, 280, 4882, 5, 78, 34, 84, 16, 3637, 3, 85, 578, 114, 573, 3, 130, 16, 4, 221, 538, 30, 39, 722, 4, 421, 655, 198, 2087, 91, 357, 2, 302, 28, 185, 469, 27, 4, 51, 254, 2984, 34, 271, 4, 4480, 37, 1185, 7371, 274, 172, 566, 3, 54, 43, 233, 100, 50, 8, 121, 48, 37, 4] 1


In [10]:
#Read test sentences and generate target y
sentences_tst_pos = read_sentences(data_path+'test/pos/')
sentences_tst_neg = read_sentences(data_path+'test/neg/')

test_x_pos = generate_sequence(tokenize(sentences_tst_pos), worddict)
test_x_neg = generate_sequence(tokenize(sentences_tst_neg), worddict)
X_test_full = test_x_pos + test_x_neg
y_test_full = [1] * len(test_x_pos) + [0] * len(test_x_neg)

print(X_test_full[0])
print(y_test_full[0])

Tokenizing...
Done!
Tokenizing...
Done!
[18877, 28, 2, 4161, 5172, 49, 32, 4135, 18, 6793, 31, 5, 307, 2166, 2333, 27, 9, 541, 4, 118, 38, 12828, 17692, 28, 1988, 11559, 27, 225, 38, 1315, 5, 28, 809, 32, 25331, 24, 35133, 31, 27, 48544, 104, 4, 472, 3, 165, 2367, 119, 3874, 8, 98, 97, 3, 13123, 9, 575, 5, 56, 52, 1490, 295, 97, 198, 5172, 41, 6850, 48, 830, 14, 146, 5, 7709, 6, 356, 388, 28, 1854, 1, 3, 11293, 21656, 27, 8, 218, 112, 8, 3414, 18, 8463, 8, 98, 46, 30539, 8, 113, 18877, 5, 17692, 174, 101, 202, 69, 12, 13, 10, 11, 12, 13, 10, 11, 137, 189, 148, 93, 16, 1002, 4, 416, 2, 268, 132, 2333, 8, 32, 4135, 18, 6793, 31, 28, 238, 7, 72, 81, 572, 27, 15, 20, 1009, 2, 279, 3, 29, 19, 183, 20, 886, 7, 277, 4, 21, 25, 88, 30, 218, 431, 765, 24, 6, 380, 28, 765, 159, 114, 96, 16, 59, 27, 5, 2, 435, 5, 1193, 35, 183, 200, 182, 4, 478, 68, 35, 6, 184, 2559, 2178, 1599, 1359, 14, 8, 4927, 224, 235, 493, 5, 2, 26, 136, 3075, 781, 4, 21, 25, 111, 56, 6, 184, 209, 691, 36, 273, 14, 6, 235, 

## Configure sequences for a RNN model
    - Remove words with low frequency
    - Truncate / complete sequences to the same length

In [11]:
#Median length of sentences
print('Median length: ', np.median([len(x) for x in X_test_full]))

Median length:  208.0


In [12]:
max_features = 50000 # Number of most frequent words selected. the less frequent recode to 0
maxlen = 200  # cut texts after this number of words (among top max_features most common words)

In [13]:
#Select the most frequent max_features, recode others using 0
def remove_features(x):
    return [[0 if w >= max_features else w for w in sen] for sen in x]

X_train = remove_features(X_train_full)
X_test  = remove_features(X_test_full)
y_train = y_train_full
y_test = y_test_full

print(X_test[1])

[1050, 25, 3, 45, 785, 7, 6, 63, 25, 20, 68, 3, 80, 3, 177, 3, 251, 3, 5, 501, 4, 15, 20, 33, 2, 1553, 7, 86, 2482, 2, 236, 1732, 12, 13, 10, 11, 12, 13, 10, 11, 401, 910, 55, 16, 3, 9, 6, 524, 453, 26, 3, 29, 15, 436, 16, 64, 93, 131, 259, 453, 4205, 12, 13, 10, 11, 12, 13, 10, 11, 0, 1, 310, 1487, 6994, 3, 47, 9, 1995, 23, 38, 12656, 336, 4, 472, 6, 619, 14, 2, 12619, 5, 126, 57, 27155, 12, 13, 10, 11, 12, 13, 10, 11, 4706, 23131, 9, 334, 5, 940, 6, 993, 257, 4, 263, 9, 46, 350, 197, 8, 2, 856, 12, 13, 10, 11, 12, 13, 10, 11, 15, 436, 2, 9757, 7860, 833, 2, 155, 3, 83, 181, 100, 139, 17, 2, 305, 123, 71, 39, 587, 3, 100, 3668, 342, 346, 49, 38, 246, 7, 710, 5, 2211, 90, 846, 9, 2, 536, 59]


In [14]:
from keras.preprocessing import sequence

# Cut or complete the sentences to length = maxlen
print("Pad sequences (samples x time)")

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print(X_test[0])

Using TensorFlow backend.


Pad sequences (samples x time)
X_train shape: (25000, 200)
X_test shape: (25000, 200)
[ 2559  2178  1599  1359    14     8  4927   224   235   493     5     2
    26   136  3075   781     4    21    25   111    56     6   184   209
   691    36   273    14     6   235    25   159     6  1061   145   476
    28  2264   492     1    27    47     9  1107     5    36   275    24
   959     5     6   717    14    72     1    56    38  4841   149    57
     8   143    38 11630   692     4   320  3469    56     6   356  2061
   119    22     6 43823    12    13    10    11    12    13    10    11
    21   134     9    63   159     1     9   220     3  2559  2273     5
  1454   125 21656    88    30    37    94     8    60    29  2647    16
   149     5  2779 12700    28     2   619     7 18877    27     5 11559
    35   682    22     2  4161  7729     4   331   537   197     9    73
     2  7729    37   438    28    60    30   949    27     5    75  1616
    24     6  5734     5    40  8981  

In [15]:
# Shuffle data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [16]:
# Export train and test data
np.save(data_path+'X_train', X_train)
np.save(data_path+'y_train', y_train)
np.save(data_path+'X_test', X_test)
np.save(data_path+'y_test', y_test)


In [18]:
# Export worddict
import pickle

with open(data_path + 'worddict.pickle', 'wb') as pfile:
    pickle.dump(worddict, pfile)
